In [31]:
import os, pandas as pd, numpy as np
import os

os.getcwd()
os.chdir("/home/parmesh/wdir/NLP/Aut_Ted_Cat_Pred_Wd_LSTM/ted_aut_txt_pred_n_clf")
df=pd.read_excel("corpus/2016_ted.xlsx")
df.head(3)

,Title,Description,Ref,TypeOfContract,Cpv (main),Cpv (additional)
0,Flexible Agreement for the Supply of Children ...,LCC is inviting suitably experienced Service P...,CS/CYP/OCE/15/296,SERVICES,85000000.0,85000000
1,Proofreading and editing services.,The EBA is seeking to establish a contract wit...,EBA/2015/07/COMM/SER/OP,SERVICES,79821100.0,NaN
2,Endoscopy Services 2.,The purpose of this tender exercise is to sele...,T15/07/RW6 (Due North DN107715),SERVICES,85121251.0,85111100


In [32]:
df = df[["Title","Cpv (main)"]]
df = df.rename(columns={"Cpv (main)":"CpvMain"})
df.head(3)

,Title,CpvMain
0,Flexible Agreement for the Supply of Children ...,85000000.0
1,Proofreading and editing services.,79821100.0
2,Endoscopy Services 2.,85121251.0


In [33]:
set(df["CpvMain"])
from collections import Counter
Counter(df["CpvMain"])

Counter({85000000.0: 834,
         79821100.0: 4,
         85121251.0: 3,
         98363000.0: 2,
         70000000.0: 38,
         85111800.0: 22,
         90500000.0: 113,
         72000000.0: 330,
         77230000.0: 2,
         64200000.0: 22,
         45251120.0: 6,
         73210000.0: 47,
         45212000.0: 12,
         48311000.0: 5,
         55524000.0: 141,
         90919300.0: 101,
         79411100.0: 21,
         45000000.0: 358,
         44111000.0: 23,
         90910000.0: 76,
         90511000.0: 6,
         45214000.0: 5,
         45350000.0: 10,
         77314000.0: 65,
         45310000.0: 21,
         34996300.0: 2,
         3413000.0: 7,
         85121230.0: 1,
         66171000.0: 10,
         98341000.0: 3,
         85311000.0: 38,
         31680000.0: 4,
         38000000.0: 164,
         48000000.0: 226,
         34100000.0: 37,
         85320000.0: 23,
         92512000.0: 2,
         16500000.0: 2,
         38510000.0: 18,
         50110000.0: 11,
        

In [34]:
df.shape

(15163, 2)

In [35]:
df = df.dropna()
df.shape

(15158, 2)

In [36]:
# Load library
from nltk.corpus import stopwords

# You will have to download the set of stop words the first time
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/parmesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
#pre-processing
import re 
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\n", "", string)    
    string = re.sub(r"\r", "", string) 
    string = re.sub(r"[0-9]", "digit", string)
    string = re.sub(r"\'", "", string)    
    return string.strip().lower()


In [38]:
X = df["Title"].astype(str).apply(clean_str)
y = np.array(df["CpvMain"])

In [40]:
def remove_stopwords(text):
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    text = pattern.sub('', text)
    return text

In [41]:
X = df["Title"].astype(str).apply(remove_stopwords)


In [42]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5)

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier

In [44]:
#preparing the final pipeline using the selected parameters
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(use_idf=True)),
    ('clf', OneVsRestClassifier(LinearSVC(class_weight="balanced")))])


In [45]:
#fit model with training data
model.fit(X_train, y_train)

/home/parmesh/anaconda3/envs/Py36/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabula...)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                         class_weight='balanced',
     

In [46]:
#evaluation on test data
pred = model.predict(X_test)

In [47]:
from sklearn.metrics import confusion_matrix, accuracy_score
conf_mat = confusion_matrix(y_test,pred )
print(conf_mat)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [49]:
accuracy_score(y_test, pred)

0.5015391380826737

In [50]:
#save the model
from sklearn.externals import joblib
joblib.dump(model, 'checkpoints/ted_2016_clf.pkl', compress=1)


/home/parmesh/anaconda3/envs/Py36/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['checkpoints/ted_2016_clf.pkl']

In [51]:
Title = input()

Management of occupational safety and health in European workplaces — evidence from the second European survey of enterprises on new and emerging risks (Esener-2).


In [52]:
model.predict([Title])[0]

73210000.0